In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Downloading required packages
!pip install -q accelerate==0.21.0  transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [3]:
#Importing the processed dataset
import pandas as pd
dataset = pd.read_excel("./drive/MyDrive/positive_negative/data.xlsx")

In [4]:
dataset.head()

,Sentence_from_X,Matched_Sentence_from_Y
0,When I lived in Los Angeles a year and a half ...,This place features everything you can expect ...
1,"I've always liked PF Chang's and, generally sp...",Right off the bat I want to admit that when it...
2,This is my 2nd time here and I thought that I ...,What is it about the anonymous Chinese buffet ...
3,"I love Whole Foods, and I have been a fan for ...",I am a big fan of the Whole Foods company in g...
4,TOTALLY one of the best restaurants in Pittsb...,"I wasn't really a fan. In fact, this may hae b..."


In [5]:
len(dataset)

893

In [6]:
dataset["Matched_Sentence_from_Y"][3]

"I am a big fan of the Whole Foods company in general.  The store back home (Omaha) had a wine bar, espresso bar, gelato, sushi and udon stand, a GIANT cheese counter where you could sample...I could go on.\\n\\nThe Pittsburgh Whole Foods is small, and almost as overcrowded as its parking lot.  Two-thirds of the time they are out of what I came for.  I'm sorry, but if I'm paying THAT much for groceries, I want a stellar shopping experience.  This WF is mediocre at best."

In [7]:
# Identify rows with NaN values
rows_with_nan = dataset[dataset.isna().any(axis=1)]

# Print rows with NaN values
print("Rows with NaN values:")
print(rows_with_nan)

Rows with NaN values:
    Sentence_from_X                            Matched_Sentence_from_Y
285             NaN  [3 Stars by PHX comparison and 2 overall] \n\n...


In [8]:
dataset = dataset.dropna()

In [9]:
# Identify rows with NaN values
rows_with_nan = dataset[dataset.isna().any(axis=1)]

# Print rows with NaN values
print("Rows with NaN values:")
print(rows_with_nan)

Rows with NaN values:
Empty DataFrame
Columns: [Sentence_from_X, Matched_Sentence_from_Y]
Index: []


In [10]:
import unicodedata
import re
import numpy as np
import os
import io
import time
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  if type(w) != str:
    print(w)
    return ""

  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  w = re.sub(r"([?.!,¿:;])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r'\\n'," ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",","¿",":",";")
  w = re.sub(r"[^a-zA-Z?.!,¿:;]+", " ", w)

    # remove extra space
  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  return w

In [11]:
print(preprocess_sentence(dataset["Sentence_from_X"][3]))

i love whole foods , and i have been a fan for a few years now . i ve visited whole foods locations around the country , but this pittsburgh location is one of my favorites . it s a small store , but it feels homey and comfortable . staff : most of the staff members are super friendly and willing to do just about anything to help you find what you are looking for . i had one staff member begrudgingly help me out , but i assumed he was having a bad day or was very busy . either way , it s nice to treat others with respect . the cashiers here are so fantastic . they are never rushing and throwing food into bags . they take their time , ask you questions and treat you like a person instead of another number in a long line . deli made to order : the deli and made to order foods are currently my favorite . sometimes i would rather spend on lunch here with my boyfriend than go to a restaurant or get fast food . my favorite is the vegan chicken salad on foccacia bread . i adore this bread . i

In [12]:
print(preprocess_sentence(dataset["Matched_Sentence_from_Y"][3]))

i am a big fan of the whole foods company in general . the store back home omaha had a wine bar , espresso bar , gelato , sushi and udon stand , a giant cheese counter where you could sample . . . i could go on . the pittsburgh whole foods is small , and almost as overcrowded as its parking lot . two thirds of the time they are out of what i came for . i m sorry , but if i m paying that much for groceries , i want a stellar shopping experience . this wf is mediocre at best .


In [13]:
print(preprocess_sentence(dataset["Matched_Sentence_from_Y"][3]).encode("UTF-8"))

b'i am a big fan of the whole foods company in general . the store back home omaha had a wine bar , espresso bar , gelato , sushi and udon stand , a giant cheese counter where you could sample . . . i could go on . the pittsburgh whole foods is small , and almost as overcrowded as its parking lot . two thirds of the time they are out of what i came for . i m sorry , but if i m paying that much for groceries , i want a stellar shopping experience . this wf is mediocre at best .'


In [14]:
#Cleaning the dataset and creating two different rows of positive and negative sentence
def create_dataset(data):
  positive_sent = [preprocess_sentence(positive) for positive in data['Sentence_from_X']]
  negative_sent = [preprocess_sentence(negative) for negative in data['Matched_Sentence_from_Y']]
  return positive_sent,negative_sent
positive_sent, negative_sent = create_dataset(dataset)

In [15]:
dataset.head()

,Sentence_from_X,Matched_Sentence_from_Y
0,When I lived in Los Angeles a year and a half ...,This place features everything you can expect ...
1,"I've always liked PF Chang's and, generally sp...",Right off the bat I want to admit that when it...
2,This is my 2nd time here and I thought that I ...,What is it about the anonymous Chinese buffet ...
3,"I love Whole Foods, and I have been a fan for ...",I am a big fan of the Whole Foods company in g...
4,TOTALLY one of the best restaurants in Pittsb...,"I wasn't really a fan. In fact, this may hae b..."


In [16]:
from sklearn.model_selection import train_test_split

# Load model directly
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

#Using the t5-small model as it is resource effective
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
train_inputs, test_inputs, train_targets, test_targets = train_test_split(positive_sent, negative_sent, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:
max_source_length = max(len(og) for og in positive_sent)
max_target_length = max(len(t) for t in negative_sent)
print(max_source_length,max_target_length)

4076 3862


In [18]:
a = ""
l = 0
for og in positive_sent:
  if len(og)>l:
    l = len(og)
    a = og
print(l)
print(og)

4076
always see this restaurant when i m grocery shopping at smiths . curiosity kicked in and wanted to try . outside appearance was not appealing . i almost didn t go in . but before i was gona leave i decided to depend on my yelp friends . it seemed to have some good reviews . first impression was divey . really needs some help . decor is old and the place is small . however the lady was very nice . i ordered chicken pad thai from lunch special . the menu says you get soup . then she nicely tells me you only get soup if you done in . she was nice and offered me done while i wait for my food . maybe she heard my stomach growl . lol . . my friend ordered orange chicken . also from lunch special . was happy when i got the bill . it was for both meals . now that s what i call a lunch special when i got home . i noticed they gave decent proportion sizes . my chicken pad thai was delicious . my friends orange chicken was good but the sauce was more runny than what i m use to seeing . didn 

In [19]:
#Defining a class to tokenize the sentence creating input for positive sentence and labels for target negative sentence
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, inputs, targets):
        self.tokenizer = tokenizer
        self.input_texts = inputs
        self.target_texts = targets
        self.max_seq_length = 512  # Maximum sequence length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]
        task_prefix = "Change Positive Sentence to Negative Sentence: "

        # Tokenize input text
        inputs = self.tokenizer(
            task_prefix + input_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_seq_length,
            return_tensors='pt',
        )

        # Tokenize target text
        labels = self.tokenizer(
            target_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_seq_length,
            return_tensors='pt',
        )

        # Extract input_ids and attention_mask
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        # Extract labels
        labels = labels['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

train_dataset = TranslationDataset(tokenizer,train_inputs,train_targets)
test_dataset = TranslationDataset(tokenizer,test_inputs,test_targets)

In [20]:
print(train_dataset.__getitem__)

<bound method TranslationDataset.__getitem__ of <__main__.TranslationDataset object at 0x78be6416f7f0>>


In [21]:
print("Maximum tokenizer length:", tokenizer.model_max_length)

Maximum tokenizer length: 512


In [22]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [23]:
from transformers import EarlyStoppingCallback
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    evaluation_strategy='steps',  # Evaluate every `eval_steps` steps
    eval_steps=100,
    fp16=True
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
# Train the model in training mode
model.train()
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
100,No log,8.330569
200,No log,1.380607
300,No log,1.209597
400,No log,1.142470
500,4.123300,1.105861
600,4.123300,1.080464
700,4.123300,1.063589
800,4.123300,1.046721
900,4.123300,1.033527
1000,1.124600,1.021798


TrainOutput(global_step=9000, training_loss=1.0378641086154514, metrics={'train_runtime': 3737.1551, 'train_samples_per_second': 19.079, 'train_steps_per_second': 2.408, 'total_flos': 9649870444953600.0, 'train_loss': 1.0378641086154514, 'epoch': 100.0})

In [24]:
model_save_path = './drive/My Drive/positive_negative/t5-pos_neg_latest'

# Make sure to create the directory if it doesn't exist
import os
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Now save the model
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./drive/My Drive/positive_negative/t5-pos_neg_latest/tokenizer_config.json',
 './drive/My Drive/positive_negative/t5-pos_neg_latest/special_tokens_map.json',
 './drive/My Drive/positive_negative/t5-pos_neg_latest/spiece.model',
 './drive/My Drive/positive_negative/t5-pos_neg_latest/added_tokens.json')

In [26]:
#Testing with different set of sentences
print("--------------Iteration 1----------")
pos_sent=["I love the korean buffet.","pizza is pretty good ! the staff is very nice , and good service ."]
model.eval()
for i in pos_sent:
  task_prefix = "Change Positive Sentence to Negative Sentence: "
  input_ids = tokenizer(
      task_prefix + i, max_length=512, padding=True, truncation=True, return_tensors="pt"
  )
  # Move tensors to the GPU if available
  input_ids = input_ids.input_ids.to(device)
  outputs = model.generate(input_ids=input_ids)
  print("Positive Input---",i)
  print("Negative Output---",tokenizer.decode(outputs[0], skip_special_tokens=True))

--------------Iteration 1----------
Positive Input--- I love the korean buffet.
Negative Output--- i love the buffet korean buffet. the food is mediocre
Positive Input--- pizza is pretty good ! the staff is very nice , and good service .
Negative Output--- pizza is good, but not good. service excellent.
